In [27]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import levene
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from itertools import combinations
import scipy.stats as stats
import pickle
import scipy.cluster.hierarchy as sch
%matplotlib notebook

In [2]:
data1 = pd.read_excel("C题数据.xlsx", "datat1").drop(labels=50).drop(labels=["组号", "编号"], axis=1)
data2 = pd.read_excel("C题数据.xlsx", "data2").drop(labels=[0, 1, 2, 8]).drop(labels=["Unnamed: 0", "Unnamed: 1"], axis=1)
data2.index = [20, 25, 30, 35, 40]
data3 = pd.read_excel("C题数据.xlsx", "data3")
data1

,厚度mm,孔隙率（%）,压缩回弹性率（%）,过滤阻力Pa,过滤效率（%）,透气性 mm/s,插层率（%）
0,1.715,93.52,77.84,8.130,4.967,777.10,NaN
1,2.810,96.28,83.20,7.533,19.967,1019.67,36.44
2,1.830,93.93,86.23,10.470,1.933,795.57,NaN
3,2.910,96.41,86.65,7.200,24.967,968.63,24.74
4,1.890,94.12,82.12,11.870,4.300,564.93,NaN
5,3.425,96.95,94.33,10.133,34.600,643.40,31.45
6,2.095,94.70,83.01,13.900,11.767,474.50,NaN
7,3.400,96.93,82.88,10.600,33.900,603.17,19.37
8,2.235,95.03,86.04,19.230,20.767,347.23,NaN
9,3.845,97.30,75.97,15.700,54.500,405.83,31.19


In [3]:
# 第一问
yv = data1.values
yi = np.delete(yv[[i for i in range(0, 50, 2)]], -1, axis=1)
yip = np.delete(yv[[i for i in range(1, 50, 2)]], -1, axis=1)
e = (yip - yi) / yi
e = np.insert(e, 6, data1["插层率（%）"].values[~np.isnan(data1["插层率（%）"].values)],axis=1)
dfe = pd.DataFrame(e, columns=data1.columns)
dfe.index = [i+1 for i in dfe.index]
dfe.index.name = "组别"
dfe.columns = ["厚度相对变化量", "孔隙率相对变化量", "压缩回弹性率相对变化量", "过滤阻力相对变化量", "过滤效率相对变化量", "透气性相对变化量", "插层率（%）"]
dfe.to_excel("相对变化量表.xlsx")
dfe

,厚度相对变化量,孔隙率相对变化量,压缩回弹性率相对变化量,过滤阻力相对变化量,过滤效率相对变化量,透气性相对变化量,插层率（%）
组别,,,,,,,
1,0.638484,0.029512,0.068859,-0.073432,3.019932,0.312148,36.44
2,0.590164,0.026403,0.004871,-0.312321,11.916192,0.217530,24.74
3,0.812169,0.030068,0.148685,-0.146335,7.046512,0.138902,31.45
4,0.622912,0.023548,-0.001566,-0.237410,1.880938,0.271170,19.37
5,0.720358,0.023887,-0.117039,-0.183567,1.624356,0.168764,31.19
6,0.449640,0.030540,-0.001700,-0.168875,14.233699,-0.037421,16.53
7,0.612903,0.034446,-0.080899,-0.007021,3.184353,0.346190,19.73
8,1.041935,0.041689,-0.002012,-0.228687,1.336823,0.252107,21.72
9,0.467363,0.022081,0.230200,-0.211804,0.348575,0.226537,18.62


In [4]:
plt.rc('font', size=14)
plt.rc('font', family="SimHei")
plt.rc('axes', unicode_minus=False)
plt.rc('figure',figsize=(12.8, 7.2))
plt.subplots_adjust(wspace=0.4, hspace=0.5)
i = 231
for tar in dfe.columns[:-1]:
    plt.subplot(i)
    plt.bar(np.arange(1, 26), dfe[tar].values)
    plt.xticks(np.array([1, 5, 10, 15, 20 ,25]))
    plt.xlabel("组别")
    plt.title(tar)
    i += 1
plt.savefig("./相对变化量直方图.png")
plt.show()
plt.clf()


<IPython.core.display.Javascript object>

In [5]:
with open("第一次方差分析结果.txt","w") as f:
    for tar in data1.columns[:-1]:
        x = np.tile(np.array([0, 1]), 25)
        y = data1[tar].values
        stat, p = levene(y[[i for i in range(0, 25, 2)]],
                         y[[i for i in range(1, 25, 2)]])
        if p > 0.05:
            f.write("对{0}进行检验得到的显著性为{1}，认为本次方差分析是齐的，可以进行方差分析\n".format(tar, p.round(4)))
            d = {'x': x, 'y': y}
            model1 = sm.formula.ols("y ~ C(x)", data=d).fit()
            s = sm.stats.anova_lm(model1)
            f.write(str(s)+'\n')
            f.write('='*80+'\n')


In [6]:
cs = sch.linkage([[i] for i in dfe["插层率（%）"].values], method='complete')
sch.dendrogram(cs, labels=np.arange(1, 26))
plt.yticks([])
plt.axhline(40, 0, 1, color='r')
plt.xlabel("组别")
plt.savefig("聚类图.png")
plt.show()
plt.clf()

<IPython.core.display.Javascript object>

In [7]:
x = sch.fcluster(cs, t=2, criterion='maxclust')
es = e[np.argsort(e[:, -1])]
w1_arg = np.where(x == 1)
w2_arg = np.where(x == 2)

In [8]:
with open("第二次方差分析结果.txt", "w") as f:
    i = 0
    for tar in dfe.columns[:-1]:
        y = dfe[tar].values
        stat, p = levene(y[w1_arg],
                         y[w2_arg])
        if p > 0.05:
            f.write(
                "对{0}进行检验得到的显著性为{1}，认为本次方差分析是齐的，可以进行方差分析\n".format(
                    tar, p.round(4)))
            d = {'x': x, 'y': y}
            model1 = sm.formula.ols("y ~ C(x)", data=d).fit()
            s = sm.stats.anova_lm(model1)
            f.write(str(s) + '\n')
            f.write('=' * 80 + '\n')
            plt.plot(es[:, -1].flat, es[:, i], marker='.', label=tar)
            i += 1
        else:
            f.write("对{0}进行检验得到的显著性为{1}，未通过检验\n".format(tar, p.round(4)))
    plt.xlabel("插层率（%）")
    plt.ylabel("变化比例")
    plt.legend()
    plt.savefig("相对变化量随插层率变化图.png")
    plt.show()
    plt.clf()


<IPython.core.display.Javascript object>

In [9]:
with open("第一题相关系数分析.txt", "w") as f:
    for tar in dfe.columns[:-1]:
        y = dfe[tar].values
        coef = np.corrcoef(y, dfe["插层率（%）"].values)[0, 1]
        f.write(
            "{0}与插层率间的相关系数为{1}\n".format(
                tar, coef.round(4)))


In [10]:
raw_data3 = data3.values
prob2_data = (raw_data3[[i for i in range(0, 75, 3)], : 5] + raw_data3[[
    i for i in range(1, 75, 3)], : 5] + raw_data3[[i for i in range(2, 75, 3)], : 5]) / 3 
prob2_df = pd.DataFrame(prob2_data, columns=data3.columns[: 5])
prob2_df


,接收距离(cm),热风速度(r/min),厚度mm,孔隙率（%）,压缩回弹性率（%）
0,40.0,800.0,2.759180,96.016300,84.793750
1,40.0,900.0,3.049211,96.660930,85.320390
2,40.0,1000.0,3.278972,96.909733,85.556427
3,40.0,1100.0,3.422445,96.757311,85.033277
4,40.0,1200.0,3.516909,96.721242,83.668346
5,35.0,800.0,2.402686,95.959635,86.719183
6,35.0,900.0,2.696144,96.405324,87.261357
7,35.0,1000.0,2.921548,96.516558,87.297681
8,35.0,1100.0,3.065953,96.391245,87.083865
9,35.0,1200.0,3.158709,96.691105,85.713582


In [11]:
with open("第二题线性回归模型.txt", "w") as f:
    f.write("x1:接收距离(cm), x2:热风速度(r/min)\ny1:厚度mm, y2:孔隙率（%）, y3:压缩回弹性率（%）\n")
    x1 = prob2_df['接收距离(cm)'].values
    x2 = prob2_df['热风速度(r/min)'].values
    y1 = prob2_df['厚度mm'].values
    y2 = prob2_df['孔隙率（%）'].values
    y3 = prob2_df['压缩回弹性率（%）'].values
    prob2_y1l = sm.formula.ols(
        'y ~ x1 + x2', {'x1': x1, 'x2': x2, 'y': y1}).fit()
    prob2_y2l = sm.formula.ols(
        'y ~ x1 + x2', {'x1': x1, 'x2': x2, 'y': y2}).fit()
    prob2_y3l = sm.formula.ols(
        'y ~ x1 + x2', {'x1': x1, 'x2': x2, 'y': y3}).fit()
    f.write("对于y1建立的模型为结果为\n")
    f.write(str(prob2_y1l.summary()) + '\n')
    f.write("其残差方差为"+str(prob2_y1l.mse_resid)+'\n')
    f.write("其均方误差（MSE）为"+str(prob2_y1l.mse_model)+'\n')
    f.write("对于y2建立的模型为结果为\n")
    f.write(str(prob2_y2l.summary()) + '\n')
    f.write("其残差方差为"+str(prob2_y2l.mse_resid)+'\n')
    f.write("其均方误差（MSE）为"+str(prob2_y2l.mse_model)+'\n')
    f.write("对于y3建立的模型为结果为\n")
    f.write(str(prob2_y3l.summary()) + '\n')
    f.write("其残差方差为"+str(prob2_y3l.mse_resid)+'\n')
    f.write("其均方误差（MSE）为"+str(prob2_y3l.mse_model)+'\n')


In [12]:
x1n = [38, 33, 28, 23, 38, 33, 28, 23]
x2n = [850, 950, 1150, 1250, 1250, 1150, 950, 850]
y1p = prob2_y1l.predict({'x1':x1n, 'x2':x2n}).values
y2p = prob2_y2l.predict({'x1':x1n, 'x2':x2n}).values
y3p = prob2_y3l.predict({'x1':x1n, 'x2':x2n}).values
res1 = pd.DataFrame([x1n, x2n, y1p, y2p, y3p], index=prob2_df.columns)
res1.index.name = '预测组别'
res1.to_excel("问题二线性回归模型预测值.xlsx")

In [13]:
x1 = prob2_df['接收距离(cm)'].values
x2 = prob2_df['热风速度(r/min)'].values
y1 = prob2_df['厚度mm'].values
y2 = prob2_df['孔隙率（%）'].values
y3 = prob2_df['压缩回弹性率（%）'].values
y = [y1, y2, y3]
formulas = [
    'y ~ x1 + x2 +I(x1**2) + I(x2**2)',
    'y ~ x1 * x2',
    'y ~ x1 * x2 +I(x1**2) + I(x2**2)']
models = []
s2 = []
for i in range(3):
    models.append([])
    s2.append([])
    for j in range(3):
        models[i].append(sm.formula.ols(
            formulas[j], {'x1': x1, 'x2': x2, 'y': y[i]}).fit())
        s2[i].append(models[i][j].mse_resid)
s2_df = pd.DataFrame(s2, index=['y1', 'y2', 'y3'], columns=['纯二次','交叉二次','完全二次'])
s2_df.index.name = '残差方差'
s2_df.to_excel("二次拟合效果比对.xlsx")


In [14]:
with open("第二题二次回归模型.txt", "w") as f:
    f.write("x1:接收距离(cm), x2:热风速度(r/min)\ny1:厚度mm, y2:孔隙率（%）, y3:压缩回弹性率（%）\n")
    for i in range(3):
        f.write("y{0}的完全二次回归模型分析如下：\n".format(i+1))
        f.write(str(models[i][2].summary()) + '\n')
        f.write("其残差方差为"+str(models[i][2].mse_resid)+'\n')
        f.write("其均方误差（MSE）为"+str(models[i][2].mse_model)+'\n')

In [15]:
y1p = []
y2p = []
y3p = []
for i in range(8):
    y1p.append(models[0][2].predict({'x1': x1n[i], 'x2': x2n[i]}).values[0])
    y2p.append(models[1][2].predict({'x1': x1n[i], 'x2': x2n[i]}).values[0])
    y3p.append(models[2][2].predict({'x1': x1n[i], 'x2': x2n[i]}).values[0])
    res2 = pd.DataFrame([x1n, x2n, y1p, y2p, y3p], index=prob2_df.columns)
    res2.index.name = '预测组别'
    res2.to_excel("问题二完全二次回归模型预测值.xlsx")

In [16]:
plt.rc('figure',figsize=(12.8, 3.2))
plt.subplots_adjust(wspace=0.4, hspace=0.5)
plt.subplot(131)
plt.xlabel('组别')
plt.ylabel('残差')
plt.title('厚度')
plt.axhline(0, 0, 1, color='k', linestyle='--', linewidth=1)
plt.scatter(np.arange(1, 26), prob2_y1l.resid)
plt.subplot(132)
plt.xlabel('组别')
plt.title('孔隙率')
plt.ylim(-0.7, 0.7)
plt.axhline(0, 0, 1, color='k', linestyle='--', linewidth=1)
plt.scatter(np.arange(1, 26), prob2_y2l.resid)
plt.subplot(133)
plt.xlabel('组别')
plt.title('压缩回弹性率')
plt.axhline(0, 0, 1, color='k', linestyle='--', linewidth=1)
plt.scatter(np.arange(1, 26), prob2_y3l.resid)
plt.savefig("线性模型残差图.png",bbox_inches = 'tight')
plt.show()
plt.clf()

<IPython.core.display.Javascript object>

In [17]:
plt.subplots_adjust(wspace=0.4, hspace=0.5)
plt.subplot(131)
plt.xlabel('组别')
plt.ylabel('残差')
plt.title('厚度')
plt.axhline(0, 0, 1, color='k', linestyle='--', linewidth=1)
plt.scatter(np.arange(1, 26), models[0][2].resid)
plt.subplot(132)
plt.xlabel('组别')
plt.title('孔隙率')
plt.axhline(0, 0, 1, color='k', linestyle='--', linewidth=1)
plt.scatter(np.arange(1, 26), models[1][2].resid)
plt.subplot(133)
plt.xlabel('组别')
plt.title('压缩回弹性率')
plt.axhline(0, 0, 1, color='k', linestyle='--', linewidth=1)
plt.scatter(np.arange(1, 26), models[2][2].resid)
plt.savefig("完全二次模型残差图.png",bbox_inches = 'tight')
plt.show()
plt.clf()

<IPython.core.display.Javascript object>

In [18]:
# 第三题
prob3_data = raw_data3[:, 2:]
prob3_df = pd.DataFrame(prob3_data, columns=data3.columns[2:])
prob3_df


,厚度mm,孔隙率（%）,压缩回弹性率（%）,过滤阻力Pa,过滤效率（%）,透气性 mm/s
0,2.766014,96.204429,84.763264,24.756047,48.211693,654.149177
1,2.754933,95.879200,84.969388,26.008212,51.443315,609.854645
2,2.756595,95.965270,84.648600,26.222296,49.679948,635.112175
3,3.051175,96.114384,85.315207,27.167856,48.599384,521.793019
4,3.051480,96.924714,85.365072,25.323787,45.963814,553.848607
...,...,...,...,...,...,...
70,2.328757,95.012779,86.629560,28.862761,80.012340,206.124684
71,2.328183,95.307393,85.896752,30.559225,81.993644,206.336370
72,2.422073,96.046859,85.250841,28.296636,83.095760,209.181523
73,2.402290,95.588749,85.144912,27.499004,83.201109,208.566982


In [19]:
x = stats.zscore(prob3_data[:, :3], axis=0, ddof=1)
y = stats.zscore(prob3_data[:, 3:], axis=0, ddof=1)
x_PCA = PCA().fit(x)
y_PCA = PCA().fit(y)
with open("主成分分析结果.txt", "w") as f:
    f.write("x1:厚度mm, x2:孔隙率（%）, x3:压缩回弹性率（%）\ny1:过滤阻力Pa, y2:过滤效率（%）, y3:透气性 mm/s\n")
    temp = ['自变量', '因变量']
    PCAs = [x_PCA, y_PCA]
    for i in range(2):
        f.write(temp[i] + "的主成分分析结果如下：\n")
        for j in range(3):
            f.write(
                "第{0}组主成分特征值为{3}贡献率为{1}，对应的系数为{2}\n".format(
                    j,
                    PCAs[i].explained_variance_ratio_[j],
                    PCAs[i].components_[j],
                    PCAs[i].explained_variance_[j]))


In [20]:
with open("第三题回归模型比较.txt", "w") as f:
    f.write("x1:厚度mm, x2:孔隙率（%）\ny2:过滤效率（%）, y3:透气性 mm/s\n")
    x1 = prob3_df['厚度mm'].values
    x2 = prob3_df['孔隙率（%）'].values
    y2 = prob3_df['过滤效率（%）'].values
    y3 = prob3_df['透气性 mm/s'].values
    prob3_x = sm.formula.ols(
        'x2 ~ x1 + I(x1**2)', {'x1': x1, 'x2': x2}).fit()
    prob3_xl = sm.formula.ols(
        'x2 ~ x1', {'x1': x1, 'x2': x2}).fit()
    prob3_y = sm.formula.ols(
        'y2 ~ y3 + I(y3**2) ', {'y3': y3, 'y2': y2}).fit()
    prob3_yl = sm.formula.ols(
        'y2 ~ y3', {'y3': y3, 'y2': y2}).fit()
    f.write("以x2为因变量x1为自变量建立的二次模型结果为\n")
    f.write(str(prob3_x.summary()) + '\n')
    f.write("其残差方差为"+str(prob3_x.mse_resid)+'\n')
    f.write("其均方误差（MSE）为"+str(prob3_x.mse_model)+'\n')
    f.write("以x2为因变量x1为自变量建立的线性模型结果为\n")
    f.write(str(prob3_xl.summary()) + '\n')
    f.write("其残差方差为"+str(prob3_xl.mse_resid)+'\n')
    f.write("其均方误差（MSE）为"+str(prob3_xl.mse_model)+'\n')
    f.write("以y2为因变量y3为自变量建立的二次模型为结果为\n")
    f.write(str(prob3_y.summary()) + '\n')
    f.write("其残差方差为"+str(prob3_y.mse_resid)+'\n')
    f.write("其均方误差（MSE）为"+str(prob3_y.mse_model)+'\n')
    f.write("以y2为因变量y3为自变量建立的线性模型为结果为\n")
    f.write(str(prob3_yl.summary()) + '\n')
    f.write("其残差方差为"+str(prob3_yl.mse_resid)+'\n')
    f.write("其均方误差（MSE）为"+str(prob3_yl.mse_model)+'\n')
    

In [21]:
x_t = np.arange(x1.min(), x1.max(), 0.01)
y_t = np.arange(y3.min(), y3.max(), 0.01)
plt.subplots_adjust(wspace=0.4, hspace=0.5)
plt.subplot(121)
plt.xlabel('厚度mm')
plt.ylabel('孔隙率（%）')
plt.scatter(x1, x2, label='实测')
plt.plot(x_t, prob3_xl.predict({'x1': x_t}).values, color='darkorange', linestyle='--',label='线性')
plt.plot(x_t, prob3_x.predict({'x1': x_t}).values, color='r', linestyle='-', label='二次')
plt.subplot(122)
plt.xlabel('透气性 mm/s')
plt.ylabel('过滤效率（%）')
plt.scatter(y3, y2, label='实测')
plt.plot(y_t, prob3_yl.predict({'y3': y_t}).values, color='darkorange', linestyle='--',label='线性')
plt.plot(y_t, prob3_y.predict({'y3': y_t}).values, color='r', linestyle='-', label='二次')
plt.legend(loc='best')
plt.savefig("第三题模型比较图.png", bbox_inches='tight')
plt.show()
plt.clf()


<IPython.core.display.Javascript object>

In [22]:
'''with open("偏最小二乘回归结果.txt", "w") as f:
    mu = prob3_data.mean(axis=0)
    s = prob3_data.std(axis=0, ddof=1)
    prob3_zsdata = stats.zscore(prob3_data, ddof=1)
    y = prob3_zsdata[:, 3:]
    x = prob3_zsdata[:, :3]
    f.write("开始通过交叉有效性检验确定选取的成分个数：\n")
    mse = []
    for i in range(1, 4):
        pls = PLSRegression(i)
        y_cv = cross_val_predict(pls, x, y)
        msei = mean_squared_error(y, y_cv)
        f.write("当选取的成分个数为{0}时交叉检验均方误差为{1}：\n".format(i, msei))
        mse.append(msei)
    minh = np.argmin(mse) + 1
    f.write("当均方误差最小时成分个数为{0}，故选用\n".format(minh))
    md = PLSRegression(minh).fit(x, y)
    raw_coef = md.coef_
    f.write("经过回归得到的标准化数据的回归系数为（每一列是y对x的回归系数）：\n" + str(raw_coef) + '\n')
    coef = np.zeros((4, 3))
    coef[0, :] = mu[3:] - mu[:3]/s[:3] @ raw_coef * s[3:]
    for i in range(3):
        coef[1:, i] = s[3+i]/s[:3] * raw_coef[:,i]
    f.write("经过回归得到的原始数据的回归系数为（每一列是y对x的回归系数,第一行为截距）：\n" + str(coef) + '\n')
    r2 = md.score(x,y)
    f.write("经过计算，该模型的R2评分为{0}\n".format(r2))
    mse = mean_squared_error(y*s[3:]+mu[3:], md.predict(x)*s[3:]+mu[3:])
    f.write("经过计算，该模型的MSE为{0}\n".format(mse))'''

'with open("偏最小二乘回归结果.txt", "w") as f:\n    mu = prob3_data.mean(axis=0)\n    s = prob3_data.std(axis=0, ddof=1)\n    prob3_zsdata = stats.zscore(prob3_data, ddof=1)\n    y = prob3_zsdata[:, 3:]\n    x = prob3_zsdata[:, :3]\n    f.write("开始通过交叉有效性检验确定选取的成分个数：\n")\n    mse = []\n    for i in range(1, 4):\n        pls = PLSRegression(i)\n        y_cv = cross_val_predict(pls, x, y)\n        msei = mean_squared_error(y, y_cv)\n        f.write("当选取的成分个数为{0}时交叉检验均方误差为{1}：\n".format(i, msei))\n        mse.append(msei)\n    minh = np.argmin(mse) + 1\n    f.write("当均方误差最小时成分个数为{0}，故选用\n".format(minh))\n    md = PLSRegression(minh).fit(x, y)\n    raw_coef = md.coef_\n    f.write("经过回归得到的标准化数据的回归系数为（每一列是y对x的回归系数）：\n" + str(raw_coef) + \'\n\')\n    coef = np.zeros((4, 3))\n    coef[0, :] = mu[3:] - mu[:3]/s[:3] @ raw_coef * s[3:]\n    for i in range(3):\n        coef[1:, i] = s[3+i]/s[:3] * raw_coef[:,i]\n    f.write("经过回归得到的原始数据的回归系数为（每一列是y对x的回归系数,第一行为截距）：\n" + str(coef) + \'\n\')\n    r2 = md.sc

In [23]:
y = prob3_data[:, 3:]
x = prob3_data[:, :3]
backup = [
    'x1',
    'x2',
    'x3',
    'x1*x2',
    'x2*x3',
    'x1*x3',
    'I(x1**2)',
    'I(x2**2)',
    'I(x3**2)',
    'x1*I(x2**2)',
    'x2*I(x3**2)',
    'x1*I(x3**2)',
    'x2*I(x1**2)',
    'x3*I(x1**2)',
    'x3*I(x2**2)',
    'I(x1**3)',
    'I(x2**3)',
    'I(x3**3)',
    'x1*x2*x3']
ybackup = ['y1', 'y2', 'y3']
strings = []
ybs = [None] * 3
ybmse_re = [0x7fffffff] * 3
for i in range(len(backup)):
    for l in list(combinations(backup, i + 1)):
        strings.append("+".join(l))
for s in strings:
    for i in range(3):
        md = sm.formula.ols(
            ybackup[i] + '~' + s, {ybackup[i]: y[:, i], 'x1': x[:, 0], 'x2': x[:, 1], 'x3': x[:, 2]}).fit()
        if md.mse_resid < ybmse_re[i]:
            ybmse_re[i] = md.mse_resid
            ybs[i] = ybackup[i] + '~' + s

prob3_yy = []
for i in range(3):
    prob3_yy.append(sm.formula.ols(
            ybs[i], {ybackup[i]: y[:, i], 'x1': x[:, 0], 'x2': x[:, 1], 'x3': x[:, 2]}).fit())

In [24]:
with open("第三题逐步回归结果.txt", "w") as f:
    f.write("x1:厚度mm, x2:孔隙率（%）, x3:压缩回弹性率（%）\ny1:过滤阻力Pa, y2:过滤效率（%）, y3:透气性 mm/s\n")
    f.write("对y1建立的三次模型结果为\n")
    f.write(str(prob3_yy[0].summary()) + '\n')
    f.write("其残差方差为"+str(prob3_yy[0].mse_resid)+'\n')
    f.write("其均方误差（MSE）为"+str(prob3_yy[0].mse_model)+'\n')
    f.write("对y2建立的三次模型结果为\n")
    f.write(str(prob3_yy[1].summary()) + '\n')
    f.write("其残差方差为"+str(prob3_yy[1].mse_resid)+'\n')
    f.write("其均方误差（MSE）为"+str(prob3_yy[1].mse_model)+'\n')
    f.write("对y3建立的三次模型结果为\n")
    f.write(str(prob3_yy[2].summary()) + '\n')
    f.write("其残差方差为"+str(prob3_yy[2].mse_resid)+'\n')
    f.write("其均方误差（MSE）为"+str(prob3_yy[2].mse_model)+'\n')

In [25]:
plt.rc('figure', figsize=(7.2, 6.4))
plot1 = sm.ProbPlot(prob3_yy[0].resid_pearson, fit=True)
plot1.ppplot(line='45')
plt.title('标准化残差的正态P-P图\n因变量：过滤阻力')
plt.xlabel('观测的累计概率')
plt.ylabel('期望的累计概率')
plt.savefig("过滤阻力标准化残差的正态P-P图.png", bbox_inches='tight')
plt.show()
plt.clf()
plot2 = sm.ProbPlot(prob3_yy[1].resid_pearson, fit=True)
plot2.ppplot(line='45')
plt.title('标准化残差的正态P-P图\n因变量：过滤效率')
plt.xlabel('观测的累计概率')
plt.ylabel('期望的累计概率')
plt.savefig("过滤效率标准化残差的正态P-P图.png", bbox_inches='tight')
plt.show()
plt.clf()
plot2 = sm.ProbPlot(prob3_yy[2].resid_pearson, fit=True)
plot2.ppplot(line='45')
plt.title('标准化残差的正态P-P图\n因变量：透气性')
plt.xlabel('观测的累计概率')
plt.ylabel('期望的累计概率')
plt.savefig("透气性标准化残差的正态P-P图.png", bbox_inches='tight')
plt.show()
plt.clf()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
with open ("第三题逐步回归模型.model", 'wb') as f: #打开文件
    pickle.dump(prob3_yy, f)

In [37]:
for i in range(3):
    pred_y=prob3_yy[i].predict({'x1':x[:,0],'x2':x[:,1],'x3':x[:,2]})
    plt.rc('figure',figsize=(12.8, 3.2))
    plt.plot(np.arange(75), y[:,i],label='y'+str(i+1))
    plt.plot(np.arange(75), pred_y,label='pred y'+str(i+1))
    plt.legend(loc='best')
    plt.savefig("y{0}比较.png".format(i+1), bbox_inches='tight')
    plt.show()
    plt.clf()

<IPython.core.display.Javascript object>